In [50]:
import pandas as pd
import numpy as np
from math import log2

In [63]:
dictCollAllSame = [{"color" : "black", "shape" : "circle", "Token Id" : tokenNumber} for tokenNumber in range(51, 100)]

In [64]:
dictCollDifferentColor = [{"color" : str(tokenNumber+100), "shape" : "square", "Token Id" : tokenNumber} for tokenNumber in range(1, 51)]

In [65]:
dictUnique = [{"color" : "black", "shape" : "triangle", "Token Id" : 100}]

In [66]:
data = pd.DataFrame(dictCollDifferentColor + dictCollAllSame +dictUnique)

In [67]:
data

,color,shape,Token Id
0,101,square,1
1,102,square,2
2,103,square,3
3,104,square,4
4,105,square,5
...,...,...,...
95,black,circle,96
96,black,circle,97
97,black,circle,98
98,black,circle,99


In [68]:
def rarityToolsScore(traits_data):
    tokenIdColumn = traits_data['Token Id']
    
    try:
        traits_data = traits_data.drop('Token ID', axis=1)
    except:
        pass 
    
    traits_data = traits_data.drop('Token Id', axis=1)
    
    if not('Trait count' in traits_data.columns):
        traits_data['Trait count'] = traits_data.count(axis=1)
    
    traits_data = traits_data.replace(np.nan, "None")
    rarity_values = {}
    for label in traits_data.columns:
        trait_counts = traits_data[label].value_counts()
        total_count = len(traits_data)
        rarity = 1 / (trait_counts / total_count)
        rarity_values[label + ' rarity score'] = traits_data[label].map(rarity)

    rarityScores = pd.DataFrame(rarity_values)
    rarityScores['Rarity score'] = rarityScores.sum(axis="columns")
    rarityScores['Token Id'] = tokenIdColumn
    return rarityScores

In [69]:
def openRarityScore(traitsData):
    tokenIdColumn = traitsData['Token Id']
    
    try:
        traitsData = traitsData.drop('Token ID', axis=1)
    except:
        pass 
    
    traitsData = traitsData.drop('Token Id', axis=1)
    
    if not('Trait count' in traitsData.columns):
        traitsData['Trait count'] = traitsData.count(axis=1)
    
    traitsData = traitsData.replace(np.nan, "None")
    traitsData['Unique'] = 0
    informationValues = {}
    for label in traitsData.drop('Unique', axis=1).columns:
      
        traitsCount = traitsData[label].value_counts()
        totalCount = len(traitsData[label])
#         print(traitsCount)
        # unique values for the column
        uniqueValues = traitsCount[traitsCount==1].index.to_list()
        # indexes with these unqiue values
        uniqueIndexes = traitsData[traitsData[label].isin(uniqueValues)].index.to_list()
        traitsData.loc[uniqueIndexes, 'Unique'] += 1
        
        prob = traitsCount / totalCount
        logProb = (1/prob).apply(log2)
        entrophy =sum(prob*logProb)
        print(entrophy)
#         informationValues[label + ' information value'] = traitsData[label].map(logProb/entrophy)
        informationValues[label + ' information value'] = traitsData[label].map(logProb)
#     display(traitsData)
    rarityScores = pd.DataFrame(informationValues)
    rarityScores['Unique'] = traitsData['Unique']
    rarityScores['Rarity score'] = rarityScores.sum(axis="columns")
    rarityScores['Token Id'] = tokenIdColumn
    
    return rarityScores



In [70]:
def openRarityScore(traitsData):
    tokenIdColumn = traitsData['Token Id']
    
    try:
        traitsData = traitsData.drop('Token ID', axis=1)
    except:
        pass 
    
    traitsData = traitsData.drop('Token Id', axis=1)
    
    if not('Trait count' in traitsData.columns):
        traitsData['Trait count'] = traitsData.count(axis=1)
    
    traitsData = traitsData.replace(np.nan, "None")
    traitsData['Unique'] = 0
    informationValues = {}
    enthropy = 0
    for label in traitsData.drop('Unique', axis=1).columns:
        traitsCount = traitsData[label].value_counts()
        totalCount = len(traitsData[label])
#         print(traitsCount)
        # unique values for the column
        uniqueValues = traitsCount[traitsCount==1].index.to_list()
        # indexes with these unqiue values
        uniqueIndexes = traitsData[traitsData[label].isin(uniqueValues)].index.to_list()
        traitsData.loc[uniqueIndexes, 'Unique'] += 1
        
        prob = traitsCount / totalCount
        logProb = (1/prob).apply(log2)
        enthropy += (prob*logProb).sum()
        informationValues[label + ' information value'] = traitsData[label].map(logProb)
#     display(traitsData)
    rarityScores = pd.DataFrame(informationValues)
    rarityScores = rarityScores.multiply(1/enthropy)
    rarityScores['Unique'] = traitsData['Unique']
    rarityScores['Rarity score'] = rarityScores.sum(axis="columns")
    rarityScores['Token Id'] = tokenIdColumn
    print(enthropy)
    return rarityScores

In [71]:
openRarityScore(data)

4.892648366158272


,color information value,shape information value,Trait count information value,Unique,Rarity score,Token Id
0,1.357926,0.204388,0.0,1,2.562315,1
1,1.357926,0.204388,0.0,1,2.562315,2
2,1.357926,0.204388,0.0,1,2.562315,3
3,1.357926,0.204388,0.0,1,2.562315,4
4,1.357926,0.204388,0.0,1,2.562315,5
...,...,...,...,...,...,...
95,0.204388,0.210345,0.0,0,0.414734,96
96,0.204388,0.210345,0.0,0,0.414734,97
97,0.204388,0.210345,0.0,0,0.414734,98
98,0.204388,0.210345,0.0,0,0.414734,99


In [72]:
0.961172 /3.7509192653692605

0.2562497169358235

In [73]:
a = openRarityScore(data)

4.892648366158272


In [ ]:
def caclulateScores(traitsData):
  tokenNumber = len(traitsData)
  traitScores = []
  for label in traitsData.columns:
    scores = []
    print('---------\n' + label+'\n')
    scoresLabels = traitsData[label].value_counts().index.tolist()
    traitsCount = traitsData[label].value_counts()
    
    for i in range(len(traitsCount)):
      scoresForI = []
      for j in range(len(traitsCount)):
        if i != j:
          scoresForI.append(traitsCount[j] / (traitsCount[i] + traitsCount[j]))
      # if we need -1/2
      scoresForI = np.array(scoresForI)
      scores.append(np.average(scoresForI-1/2))
      # scores.append(np.average(scoresForI))
    normalization = np.sum(np.multiply(scores, traitsCount)) / np.sum(traitsCount)
    scores /= normalization
    # scores = scores.tolist()
    traitScores.append(dict(zip(scoresLabels, scores)))
  return traitScores

In [80]:
def openRarityScore(traitsData):
    tokenIdColumn = traitsData['Token Id']
    
    try:
        traitsData = traitsData.drop('Token ID', axis=1)
    except:
        pass 
    
    traitsData = traitsData.drop('Token Id', axis=1)
    
    if not('Trait count' in traitsData.columns):
        traitsData['Trait count'] = traitsData.count(axis=1)
    
    traitsData = traitsData.replace(np.nan, "None")
    traitsData['Unique'] = 0
    informationValues = {}
    enthropy = 0
    for label in traitsData.drop('Unique', axis=1).columns:
        traitsCount = traitsData[label].value_counts()
        totalCount = len(traitsData[label])
#         print(traitsCount)
        # unique values for the column
        uniqueValues = traitsCount[traitsCount==1].index.to_list()
        # indexes with these unqiue values
        uniqueIndexes = traitsData[traitsData[label].isin(uniqueValues)].index.to_list()
        traitsData.loc[uniqueIndexes, 'Unique'] += 1
        
        prob = traitsCount / totalCount
        logProb = (1/prob).apply(log2)
        display(pd.DataFrame(logProb))
        enthropy += (prob*logProb).sum()
        informationValues[label + ' information value'] = traitsData[label].map(logProb)
#     display(traitsData)
    rarityScores = pd.DataFrame(informationValues)
    rarityScores = rarityScores.multiply(1/enthropy)
    rarityScores['Unique'] = traitsData['Unique']
    rarityScores['Rarity score'] = rarityScores.sum(axis="columns")
    rarityScores['Token Id'] = tokenIdColumn
    print(enthropy)
    return rarityScores

In [81]:
openRarityScore(data)

,color
black,1.000000
139,6.643856
129,6.643856
130,6.643856
131,6.643856
132,6.643856
133,6.643856
134,6.643856
135,6.643856
136,6.643856


,shape
square,1.000000
circle,1.029146
triangle,6.643856


,Trait count
2,0.0


4.892648366158272


,color information value,shape information value,Trait count information value,Unique,Rarity score,Token Id
0,1.357926,0.204388,0.0,1,2.562315,1
1,1.357926,0.204388,0.0,1,2.562315,2
2,1.357926,0.204388,0.0,1,2.562315,3
3,1.357926,0.204388,0.0,1,2.562315,4
4,1.357926,0.204388,0.0,1,2.562315,5
...,...,...,...,...,...,...
95,0.204388,0.210345,0.0,0,0.414734,96
96,0.204388,0.210345,0.0,0,0.414734,97
97,0.204388,0.210345,0.0,0,0.414734,98
98,0.204388,0.210345,0.0,0,0.414734,99


In [101]:
6.643856 / 3.7509192653692605

1.7712607310266764

In [ ]:
0.961172 / 

In [ ]:
def caclulateScores(traitsData):
  tokenNumber = len(traitsData)
  traitScores = []
  for label in traitsData.columns:
    scores = []
    print('---------\n' + label+'\n')
    scoresLabels = traitsData[label].value_counts().index.tolist()
    traitsCount = traitsData[label].value_counts()
    
    for i in range(len(traitsCount)):
      scoresForI = []
      for j in range(len(traitsCount)):
        if i != j:
          scoresForI.append(traitsCount[j] / (traitsCount[i] + traitsCount[j]))
      # if we need -1/2
      scoresForI = np.array(scoresForI)
      scores.append(np.average(scoresForI-1/2))
      # scores.append(np.average(scoresForI))
    normalization = np.sum(np.multiply(scores, traitsCount)) / np.sum(traitsCount)
    scores /= normalization
    # scores = scores.tolist()
    traitScores.append(dict(zip(scoresLabels, scores)))
  return traitScores

In [87]:
def tournamentScore(traitsData):
    tokenIdColumn = traitsData['Token Id']
    
    try:
        traitsData = traitsData.draop('Token ID', axis=1)
    except:
        pass 
    
    traitsData = traitsData.drop('Token Id', axis=1)
        
    traitsData = traitsData.replace(np.nan, "None")
    tokenNumber = len(traitsData)
    rarityValues = {}
    
    for label in traitsData.columns:
        print('------------'+label+'------------\n\n')
        scores = []
        scoresLabels = traitsData[label].value_counts().index.tolist()
        traitsCount = traitsData[label].value_counts()
        
        for i in range(len(traitsCount)):
            scoresForI = []
            if len(traitsCount) == 1:
                  scoresForI.append(0)
            for j in range(len(traitsCount)):

                if i != j:
                    scoresForI.append(traitsCount[j] / (traitsCount[i] + traitsCount[j]))
            scoresForI = np.array(scoresForI)
            scores.append(np.average(scoresForI))
#             display(pd.DataFrame(scores))
        normalization = np.sum(np.multiply(scores, traitsCount)) / np.sum(traitsCount)
        print(normalization)
        if normalization != 0:
            scores /= normalization
        rarityValues[label] = traitsData[label].map(dict(zip(scoresLabels, scores)))
    
    rarityScores = pd.DataFrame(rarityValues)
    rarityScores['Rarity score'] = rarityScores.sum(axis="columns")
    rarityScores['Token Id'] = tokenIdColumn
    return rarityScores

In [89]:
tournamentScore(data).sort_values('Rarity score', ascending=False).head(52)

------------color------------


0.2646078431372549
------------shape------------


0.26707866904337496


,color,shape,Rarity score,Token Id
99,0.074102,3.670065,3.744167,100
26,1.925898,0.963307,2.889205,27
28,1.925898,0.963307,2.889205,29
29,1.925898,0.963307,2.889205,30
30,1.925898,0.963307,2.889205,31
31,1.925898,0.963307,2.889205,32
32,1.925898,0.963307,2.889205,33
33,1.925898,0.963307,2.889205,34
34,1.925898,0.963307,2.889205,35
35,1.925898,0.963307,2.889205,36


In [83]:
rarityToolsScore(data).sort_values('Rarity score', ascending=False).head(52)

,color rarity score,shape rarity score,Trait count rarity score,Rarity score,Token Id
0,100.0,2.000000,1.0,103.000000,1
26,100.0,2.000000,1.0,103.000000,27
28,100.0,2.000000,1.0,103.000000,29
29,100.0,2.000000,1.0,103.000000,30
30,100.0,2.000000,1.0,103.000000,31
31,100.0,2.000000,1.0,103.000000,32
32,100.0,2.000000,1.0,103.000000,33
33,100.0,2.000000,1.0,103.000000,34
34,100.0,2.000000,1.0,103.000000,35
35,100.0,2.000000,1.0,103.000000,36


In [78]:
 openRarityScore(data)

4.892648366158272


,color information value,shape information value,Trait count information value,Unique,Rarity score,Token Id
0,1.357926,0.204388,0.0,1,2.562315,1
1,1.357926,0.204388,0.0,1,2.562315,2
2,1.357926,0.204388,0.0,1,2.562315,3
3,1.357926,0.204388,0.0,1,2.562315,4
4,1.357926,0.204388,0.0,1,2.562315,5
...,...,...,...,...,...,...
95,0.204388,0.210345,0.0,0,0.414734,96
96,0.204388,0.210345,0.0,0,0.414734,97
97,0.204388,0.210345,0.0,0,0.414734,98
98,0.204388,0.210345,0.0,0,0.414734,99


In [13]:
a.sort_values('Rarity score', ascending=False).head(51)

,color information value,shape information value,Trait count information value,Unique,Rarity score,Token Id
0,0.066439,0.504282,0.0,1,1.570720,51
25,0.066439,0.504282,0.0,1,1.570720,76
27,0.066439,0.504282,0.0,1,1.570720,78
28,0.066439,0.504282,0.0,1,1.570720,79
29,0.066439,0.504282,0.0,1,1.570720,80
30,0.066439,0.504282,0.0,1,1.570720,81
31,0.066439,0.504282,0.0,1,1.570720,82
32,0.066439,0.504282,0.0,1,1.570720,83
33,0.066439,0.504282,0.0,1,1.570720,84
34,0.066439,0.504282,0.0,1,1.570720,85


In [100]:
data

,color,shape,Token Id
0,151,square,51
1,152,square,52
2,153,square,53
3,154,square,54
4,155,square,55
...,...,...,...
95,black,circle,47
96,black,circle,48
97,black,circle,49
98,black,circle,50


In [5]:
color*50

['black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black']